# <u>学習済みモデルを用いて推定</u>

In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import datetime
import csv

## <font color="Red">↓ここに推定したい１行動分のデータファイルを記入する</font>

In [2]:
'''
()内に，これから入力する新規ファイルのパスを書く(入力するファイル名は何でもOK)．
現在あらかじめファイルを記入していますが，ここを変更してもらう形になります．
現在入力しているファイルは，学習時に用いなかった被験者のデータであり，行動01（腕立て伏せ）のデータなので，出力として
「腕立て伏せ」がアウトプットされていれば正しく認識できていたということになります．
'''
new_data=[]
path = os.getcwd()
## 右手首のファイル
new_data.append(path+'/data_imu/id01_s04_a01_r01.csv')
## 左手首のファイル
new_data.append(path+'/data_imu/id02_s04_a01_r01.csv')
## 右足首のファイル
new_data.append(path+'/data_imu/id03_s04_a01_r01.csv')
## 左足首のファイル
new_data.append(path+'/data_imu/id04_s04_a01_r01.csv')
## 腰の中央のファイル
new_data.append(path+'/data_imu/id05_s04_a01_r01.csv')

## 残りは全て実行するのみ

In [3]:
#imuから取得されたデータのカラム名を指定
header=[
    'index',
    'mag_x','mag_y','mag_z',
    'acc_x','acc_y','acc_z',
    'gyro_x','gyro_y','gyro_z',
    'qw','qx','qy','qz'
]

all_data=[]
for file in new_data:
    data = pd.read_csv(file, header=None, skiprows=10, names=header)
    #全てのデータファイルをall_dataに格納
    all_data.append(data)
    
    
#使用するデータ（カラム）を指定
use_data=[
    'acc_x','acc_y','acc_z',
    'gyro_x','gyro_y','gyro_z',
    'qw','qx','qy','qz'
]

#特徴量の数．最後の数字は何種の特徴量を用いるか
nfeats = len(use_data)*5
x = np.empty((0,nfeats))


#各センサごとに1ファイルずつ読み込んで特徴量抽出を行う
for data in all_data:
    #使用しないカラムを削除
    data = data.drop(['index','mag_x','mag_y','mag_z'],axis=1)
    data = data.replace('        NaN',0.0)
    data = data.replace('NaN',0.0)
    
    #特徴量抽出
    mean_data = np.mean(data, axis=0)
    std_data = np.std(data, axis=0)
    var_data = np.var(data, axis=0)
    max_data = np.max(data,axis=0)
    min_data = np.min(data,axis=0)
    
    
    features = np.empty((1,0))
    features = np.append(features, mean_data)
    features = np.append(features, std_data)
    features = np.append(features, var_data)
    features = np.append(features, max_data)
    features = np.append(features, min_data)
    
    
    #1ファイルを一行に変換する
    if len(features) == nfeats:    
        x = np.vstack((x, features.reshape(1,len(features)))) #vstackは配列を結合する
    else:
        print("error")

In [4]:
#各センサのデータを1つに結合
num_feats = nfeats*int(x.shape[0]) #最後の数字は使用するセンサの数
X = np.empty((0,num_feats))
all_sensors =[]

for i in range(len(new_data)):
    all_sensors.extend(x[i])                
X = np.vstack((X, np.array(all_sensors)))

### 学習済みモデルに入力&出力

In [5]:
#"training_model.ipynb"で学習済みモデルを読み込み.
filename = 'trained_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
y_pred = loaded_model.predict(X)
dt_now = datetime.datetime.now()

#対象行動群
actions={'01':'act01:Push Up', 'act02':'02:Crunch','03':'act03:Leg raise','04':'act04:Plank',
         '05':'act05:Bird Dog','06':'act06:Hip Lift','07':'act07:Squat','08':'act08:Front Lunge',
         '09':'act09:Running','10':'act10:Single Leg Drop Jumps','11':'act11:Continuous jumps'}

#csvファイルに出力
with open('estimation_result/'+dt_now.strftime('%Y%m%d_%H%M%S')+'.csv', 'w') as f:
    # header を設定
    fieldnames = ['date_time', 'result']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    # データの書き込み
    writer.writerow({'date_time':dt_now.strftime('%Y/%m/%d/%H:%M:%S'), 'result':actions[y_pred[0]]} )
    
print("estimation finished")

estimation finished


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## 出力結果が"estimation_result"フォルダ内に保存されます．